In [ ]:
import pandas as pd
import random
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans

In [3]:
# Load the dataset (you'll need to replace this with the actual path to your CSV file)
data = pd.read_csv('/kaggle/input/saathi-travel/travel_data_with_complex_relations_and_reviews.csv')

# One-hot encode 'vibe' feature
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
vibe_encoded = encoder.fit_transform(data[['vibe']])
vibe_df = pd.DataFrame(vibe_encoded, columns=encoder.get_feature_names_out(['vibe']))
data = pd.concat([data, vibe_df], axis=1)

# Normalize numeric features
scaler = StandardScaler()
data['expenses (INR)'] = scaler.fit_transform(data[['expenses (INR)']])

# Select features for clustering
features = data.drop(columns=['user_id', 'vibe', 'reviews', 'instagram_photos', 'previous_destinations', 'favorite_activities'])

# Apply KMeans clustering
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
data['cluster'] = kmeans.fit_predict(features)

destination_info = {
    'Agra': {'activities': ['Photography', 'Exploring', 'Cultural'], 'vibes': ['Cultural', 'Budget'], 'expense': 4000, 'type': 'city'},
    'Jaipur': {'activities': ['Photography', 'Shopping', 'Cultural'], 'vibes': ['Cultural', 'Luxury'], 'expense': 5000, 'type': 'city'},
    'Goa': {'activities': ['Beach', 'Nightlife', 'Swimming'], 'vibes': ['Beach', 'Relaxation', 'Nightlife'], 'expense': 8500, 'type': 'beach'},
    'Kerala': {'activities': ['Relaxation', 'Nature', 'Backwaters'], 'vibes': ['Nature', 'Relaxation'], 'expense': 7000, 'type': 'nature'},
    'Varanasi': {'activities': ['Cultural', 'Photography', 'Exploring'], 'vibes': ['Cultural', 'Budget'], 'expense': 3500, 'type': 'city'},
    'Mysore': {'activities': ['Cultural', 'Photography', 'Fine Dining'], 'vibes': ['Cultural', 'Luxury'], 'expense': 4500, 'type': 'city'},
    'Rishikesh': {'activities': ['Adventure', 'Hiking', 'Camping'], 'vibes': ['Adventure', 'Nature'], 'expense': 3000, 'type': 'nature'},
    'Udaipur': {'activities': ['Luxury', 'Photography', 'Fine Dining'], 'vibes': ['Luxury', 'Cultural'], 'expense': 7000, 'type': 'city'},
    'Mumbai': {'activities': ['Nightlife', 'Shopping', 'Fine Dining'], 'vibes': ['Nightlife', 'Luxury'], 'expense': 8000, 'type': 'city'},
    'Delhi': {'activities': ['Cultural', 'Shopping', 'Fine Dining'], 'vibes': ['Cultural', 'Nightlife'], 'expense': 6500, 'type': 'city'},
    'Puducherry': {'activities': ['Beach', 'Cultural', 'Relaxation'], 'vibes': ['Beach', 'Cultural'], 'expense': 5000, 'type': 'beach'},
    'Andaman and Nicobar Islands': {'activities': ['Beach', 'Swimming', 'Nature'], 'vibes': ['Beach', 'Nature', 'Adventure'], 'expense': 9000, 'type': 'beach'},
    'Manali': {'activities': ['Hiking', 'Skiing', 'Nature'], 'vibes': ['Adventure', 'Nature'], 'expense': 5500, 'type': 'nature'},
    'Shimla': {'activities': ['Hiking', 'Nature', 'Photography'], 'vibes': ['Nature', 'Relaxation'], 'expense': 6000, 'type': 'nature'},
    'Darjeeling': {'activities': ['Nature', 'Hiking', 'Photography'], 'vibes': ['Nature', 'Relaxation'], 'expense': 5500, 'type': 'nature'},
    'Leh-Ladakh': {'activities': ['Adventure', 'Photography', 'Hiking'], 'vibes': ['Adventure', 'Nature'], 'expense': 7500, 'type': 'nature'},
    'Munnar': {'activities': ['Nature', 'Hiking', 'Photography'], 'vibes': ['Nature', 'Relaxation'], 'expense': 6000, 'type': 'nature'},
    'Auli': {'activities': ['Skiing', 'Nature', 'Adventure'], 'vibes': ['Adventure', 'Nature'], 'expense': 7000, 'type': 'nature'},
    'Spiti Valley': {'activities': ['Adventure', 'Hiking', 'Photography'], 'vibes': ['Adventure', 'Nature'], 'expense': 6500, 'type': 'nature'},
    'Nainital': {'activities': ['Nature', 'Boating', 'Hiking'], 'vibes': ['Nature', 'Relaxation'], 'expense': 5500, 'type': 'nature'}
}

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
def recommend_itinerary(user_input, past_destinations, travel_mates_vibe):
    # Prepare the user input DataFrame
    user_df = pd.DataFrame([user_input])
    
    # One-hot encode the 'vibe' input
    vibe_encoded_input = encoder.transform(user_df[['vibe']])
    vibe_df_input = pd.DataFrame(vibe_encoded_input, columns=encoder.get_feature_names_out(['vibe']))
    
    # Normalize the expenses input
    user_df['expenses (INR)'] = scaler.transform(user_df[['expenses (INR)']])
    
    # Combine the encoded vibe with the normalized expenses
    user_features = pd.concat([user_df.drop(columns=['vibe']), vibe_df_input], axis=1)
    
    # Predict the user's cluster
    user_cluster = kmeans.predict(user_features)[0]
    
    # Filter the data to find users in the same cluster
    similar_users = data[data['cluster'] == user_cluster]
    
    # Generate recommendations based on the most common previous destinations and activities in this cluster
    recommended_destinations = similar_users['previous_destinations'].mode().tolist()
    
    # Choose a destination that hasn't been visited before
    recommended_destination = next((dest for dest in recommended_destinations if dest not in past_destinations), None)
    
    # If all recommended destinations have been visited, choose a random one from the destination_info
    if recommended_destination is None:
        recommended_destination = random.choice(list(destination_info.keys()))
    
    # Determine possible destinations based on travel mates' vibes
    possible_destinations = [
        dest for dest, info in destination_info.items() 
        if any(vibe in info['vibes'] for vibe in travel_mates_vibe)
    ]
    
    # If the recommended destination doesn't match travel mates' vibes, choose a random one that does
    if recommended_destination not in possible_destinations:
        recommended_destination = random.choice(possible_destinations)
    
    # Final recommendation
    itinerary = {
        'Suggested Destination': recommended_destination,
        'Recommended Activities': destination_info[recommended_destination]['activities'],
        'Average Expenses': destination_info[recommended_destination]['expense'],
    }
    
    return itinerary

In [5]:
def travel_recommendation():
    # Define possible values for vibes and previous destinations
    vibes = ['Adventure', 'Relaxation', 'Luxury', 'Budget', 'Nature', 
             'Beach', 'Cultural', 'Nightlife', 'Solo Travel']
    
    destinations = list(destination_info.keys())
    
    # Collect user input
    print("Select your vibe:")
    for i, vibe in enumerate(vibes):
        print(f"{i + 1}. {vibe}")
    selected_vibe = vibes[int(input("Enter the number corresponding to your vibe: ")) - 1]
    
    print("\nSelect your previous destination:")
    for i, destination in enumerate(destinations):
        print(f"{i + 1}. {destination}")
    prev_destination = destinations[int(input("Enter the number corresponding to your previous destination: ")) - 1]
    
    print("\nSelect your travel mate's vibe:")
    for i, vibe in enumerate(vibes):
        print(f"{i + 1}. {vibe}")
    travel_mate_vibe = vibes[int(input("Enter the number corresponding to your travel mate's vibe: ")) - 1]
    
    budget = float(input("\nEnter your budget (per day in INR): "))
    
    # Create user input dictionary for recommendations
    user_input = {
        'expenses (INR)': budget,  # User's budget
        'vibe': selected_vibe,      # Selected vibe
    }

    # Previous destinations and travel mates' vibes
    previous_destinations = [prev_destination]  # List of previous destinations
    travel_mates_vibe = [travel_mate_vibe]      # List of travel mates' vibes
    
    # Output the collected information
    print("\n--- Travel Preferences ---")
    print(f"Selected Vibe: {selected_vibe}")
    print(f"Previous Destination: {prev_destination}")
    print(f"Travel Mate's Vibe: {travel_mate_vibe}")
    print(f"Budget: {budget}")

    # Get recommendations
    itinerary = recommend_itinerary(user_input, previous_destinations, travel_mates_vibe)
    
    # Output the recommended itinerary
    print("\n--- Recommended Itinerary ---")
    for key, value in itinerary.items():
        print(f"{key}: {value}")

In [ ]:
travel_recommendation()